<a href="https://colab.research.google.com/github/Vaishu7777/datascience/blob/main/PHASE%202%20DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Data Preprocessing

STEP :1
●	import pandas as pd
●	# Load the dataset
●	df = pd.read_csv("fake_or_real_news.csv")  # Replace with your actual filename
●	df.head().]
●	STEP 2:
●	# Check for missing values
●	print(df.isnull().sum())
●	STEP 3:
●	# Check for duplicates
●	duplicates = df.duplicated().sum()
●	print(f"Duplicate rows: {duplicates}")
●
●	# Remove duplicates
●	df = df.drop_duplicates()
●
●	# Drop rows with too many missing values (if any)
●	df = df.dropna(subset=["title", "text", "label"])  # drop rows missing critical info
●
●	# Optionally fill missing with placeholders or impute if needed
●	df['author'] = df['author'].fillna("Unknown")
●	STEP 4:
●	# If you have numerical features like 'readability_score' or 'num_words', detect outliers
●	import seaborn as sns
●	import matplotlib.pyplot as plt
●
●	# Example for a feature like 'text_length'
●	df['text_length'] = df['text'].apply(lambda x: len(x.split()))
●
●	# Boxplot to detect outliers
●	sns.boxplot(x=df['text_length'])
●	plt.show()
●
●	# Remove outliers using IQR method
●	Q1 = df['text_length'].quantile(0.25)
●	Q3 = df['text_length'].quantile(0.75)
●	IQR = Q3 - Q1
●
●	df = df[(df['text_length'] >= Q1 - 1.5 * IQR) & (df['text_length'] <= Q3 + 1.5 * IQR)]
●	STEP 5:
●	# Convert date columns to datetime, if present
●	# df['date'] = pd.to_datetime(df['date'])
●
●	# Ensure labels are strings
●	df['label'] = df['label'].astype(str)
●	STEP 6: # Label Encoding (if binary labels like 'fake' and 'real')
●	df['label_encoded'] = df['label'].map({'fake': 0, 'real': 1})
●
●	# One-Hot Encoding (for multiple categories, e.g., news sources)
●	# pd.get_dummies(df['source'], prefix='src')
●	STEP 7:
●	from sklearn.preprocessing import MinMaxScaler
●
●	# Normalize 'text_length' for example
●	scaler = MinMaxScaler()
●	df['text_length_scaled'] = scaler.fit_transform(df[['text_length']])
●	STEP 8:
●	# Display cleaned dataset info
●	print(df.info())
●	df.head()


In [ ]:
#Exploratory Data Analysis (EDA)
STEP 1: import matplotlib.pyplot as plt
import seaborn as sns

# Add a new column for word count
df['text_length'] = df['text'].apply(lambda x: len(x.split()))

# Histogram of text length
plt.figure(figsize=(8,5))
sns.histplot(df['text_length'], bins=50, kde=True)
plt.title("Distribution of Text Length")
plt.xlabel("Word Count")
plt.ylabel("Frequency")
plt.show()
STEP 2:
plt.figure(figsize=(8,5))
sns.boxplot(x='label', y='text_length', data=df)
plt.title("Text Length by Label (Fake vs Real)")
plt.xlabel("Label")
plt.ylabel("Word Count")
plt.show()
STEP 3# sns.pairplot(df[['text_length', 'label_encoded', 'other_numeric_feature']], hue='label'):
STEP 4: plt.figure(figsize=(8,5))
sns.violinplot(x='label', y='text_length', data=df)
plt.title("Violin Plot: Text Length by Label")
plt.show()
STEP 5: # Assuming you have a 'sentiment' column
# sns.barplot(x='label', y='sentiment_score', data=df)


In [ ]:
1. Create New Features Based on Domain Knowledge or EDA
📌 Add text_length and title_length
python
CopyEdit
# Length of text and title
df['text_length'] = df['text'].apply(lambda x: len(x.split()))
df['title_length'] = df['title'].apply(lambda x: len(x.split()))
Justification: Fake news may use shorter, attention-grabbing titles and verbose or sensational content. These length features help capture writing style.
________________________________________
📌 Add punctuation_count and uppercase_ratio
python
CopyEdit
df['punctuation_count'] = df['text'].apply(lambda x: sum([1 for c in x if c in ".,!?;:"]))
df['uppercase_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
Justification: Fake articles often contain excessive punctuation or uppercase letters to express urgency or emotion.
________________________________________
🔄 2. Combine or Split Columns
📌 Extract Date Parts (if date column exists)
python
CopyEdit
# If there's a 'date' column
# df['date'] = pd.to_datetime(df['date'])
# df['year'] = df['date'].dt.year
# df['month'] = df['date'].dt.month
# df['dayofweek'] = df['date'].dt.dayofweek
Justification: Temporal trends may reveal fake news clustering in specific months or days (e.g., elections).
________________________________________
📊 3. Apply Binning / Grouping
📌 Bin text length into categories
python
CopyEdit
df['text_length_bin'] = pd.cut(df['text_length'], bins=[0, 300, 600, 900, 1200, np.inf], labels=['Very Short', 'Short', 'Medium', 'Long', 'Very Long'])
Justification: Helps tree-based models or visualizations by grouping similar-sized articles.
________________________________________
✨ 4. Polynomial and Interaction Features (for advanced models)
📌 Add interaction between title and text lengths
python
CopyEdit
df['length_ratio'] = df['title_length'] / (df['text_length'] + 1)
Justification: Fake news may have disproportionately short titles compared to body length, highlighting clickbait patterns.
________________________________________
🔻 5. Dimensionality Reduction (Optional)
📌 Apply PCA to TF-IDF vectors
python
CopyEdit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = vectorizer.fit_transform(df['text'])

# PCA
pca = PCA(n_components=100)
X_pca = pca.fit_transform(X_tfidf.toarray())
Justification: PCA reduces high-dimensional TF-IDF space, speeding up training and reducing overfitting.
________________________________________
✅ Final Justification Summary
Feature	Purpose	Justification
text_length	Numerical	Detect article verbosity
title_length	Numerical	Spot clickbait headlines
punctuation_count	Style indicator	Excess punctuation = urgency/sensationalism
uppercase_ratio	Tone indicator	Uppercase overuse = emotional manipulation
length_ratio	Interaction	Title-to-body ratio reflects content quality
text_length_bin	Categorical	Helps grouping similar articles
PCA components	Dimensionality	Compresses TF-IDF without losing major variance

8. Model Building
Target: label_encoded (0 = Fake, 1 = Real)
________________________________________
✅ Step 1: Import Required Libraries
python
CopyEdit
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
________________________________________
🔠 Step 2: Feature Preparation (TF-IDF on Text)
python
CopyEdit
# Vectorize text data
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(df['text'])
y = df['label_encoded']
________________________________________
✂️ Step 3: Train-Test Split (with Stratification)
python
CopyEdit
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)
________________________________________
🤖 Step 4: Model 1 – Logistic Regression
python
CopyEdit
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_lr = logreg.predict(X_test)
________________________________________
🌳 Model 2 – Random Forest Classifier
python
CopyEdit
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
________________________________________
📊 Step 5: Evaluation Metrics Function
python
CopyEdit
def evaluate_model(name, y_true, y_pred):
    print(f"\nModel: {name}")
    print(f"Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall:    {recall_score(y_true, y_pred):.4f}")
    print(f"F1-Score:  {f1_score(y_true, y_pred):.4f}")
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=['Fake', 'Real']))
________________________________________
📈 Step 6: Evaluate Both Models
python
CopyEdit
evaluate_model("Logistic Regression", y_test, y_pred_lr)
evaluate_model("Random Forest", y_test, y_pred_rf)
________________________________________
🧠 Justification of Model Choice
Model	Reason for Selection
Logistic Regression	A simple, interpretable baseline for binary classification, fast and efficient on sparse TF-IDF data.
Random Forest	A powerful ensemble method that handles non-linear patterns and gives feature importance. Robust to overfitting.
________________________________________
🔍 Example Output Summary (Hypothetical)
Model	Accuracy	Precision	Recall	F1-score
Logistic Regression	0.93	0.91	0.94	0.92
Random Forest	0.96	0.95	0.93	0.92


9. Visualization of Results & Model Insights
.] 1. Confusion Matrix (Already Done Earlier)
What it shows:
A summary of prediction results:
•	TP (True Positive): Real news predicted as real.
•	TN (True Negative): Fake news predicted as fake.
•	FP (False Positive): Fake predicted as real.
•	FN (False Negative): Real predicted as fake.
python
CopyEdit
from sklearn.metrics import ConfusionMatrixDisplay

# Repeat for XGBoost
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_xgb, display_labels=['Fake', 'Real'], cmap='Blues')
plt.title("Confusion Matrix - XGBoost")
plt.show()
🔍 Conclusion:
Fewer off-diagonal values = better performance. High TP and TN indicate strong predictive accuracy.
________________________________________
📈 2. ROC Curve and AUC Score
What it shows:
ROC (Receiver Operating Characteristic) curve plots the trade-off between true positive rate (sensitivity) and false positive rate.
python
CopyEdit
from sklearn.metrics import roc_curve, auc

# Predict probabilities
y_probs = xgb.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,4))
plt.plot(fpr, tpr, label=f'XGBoost (AUC = {roc_auc:.2f})')
plt.plot([0,1], [0,1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid()
plt.show()
🔍 Conclusion:
Higher AUC (Area Under Curve) means better separability of fake vs real news. AUC near 1.0 = excellent classifier.
________________________________________
🌳 3. Feature Importance Plot (XGBoost or Random Forest)
What it shows:
Top features the model used to make decisions. Useful for interpretability.
python
CopyEdit
import numpy as np
import seaborn as sns

# For XGBoost
importances = xgb.feature_importances_
feature_names = vectorizer.get_feature_names_out()
top_indices = np.argsort(importances)[-20:][::-1]

plt.figure(figsize=(8,6))
sns.barplot(x=importances[top_indices], y=np.array(feature_names)[top_indices])
plt.title("Top 20 Important Words (XGBoost)")
plt.xlabel("Feature Importance Score")
plt.ylabel("Word")
plt.show()
🔍 Conclusion:
Words like "breaking", "shocking", "election", or emotionally charged terms are often top predictors in fake news.
________________________________________
📊 4. Visual Comparison of Model Performance
python
CopyEdit
import pandas as pd

# Example metrics (replace with actual values)
metrics = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost'],
    'Accuracy': [0.93, 0.96, 0.97],
    'F1-Score': [0.92, 0.96, 0.97]
})

metrics.set_index('Model')[['Accuracy', 'F1-Score']].plot(kind='bar', figsize=(8,5), colormap='viridis')
plt.title('Model Comparison: Accuracy & F1 Score')
plt.ylabel('Score')
plt.ylim(0.85, 1.0)
plt.grid(axis='y')
plt.show()
